In [3]:
import os
os.environ["KERAS_BACKEND"] = "jax"
import jax
import keras
import numpy as np
import tensorflow as tf

NotFoundError: dlopen(/Users/ethanvizitei/Code/Personal/dlwp/venv/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Users/ethanvizitei/Code/Personal/dlwp/venv/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <E6BA2FD2-654D-380C-8224-59AA45D1133C> /Users/ethanvizitei/Code/Personal/dlwp/venv/lib/python3.10/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so

In [4]:
class Dataset:
    def __init__(self, data, labels, batch_size):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self._indices = np.arange(len(data))
    def __iter__(self):
        for start_idx in range(0, len(self.data), self.batch_size):
            end_idx = start_idx + self.batch_size
            batch_indices = self._indices[start_idx:end_idx]
            yield (self.data[batch_indices], self.labels[batch_indices])
    def __len__(self):
        return (len(self.data) + (self.batch_size - 1)) // self.batch_size

In [2]:
def get_model():
    inputs = keras.Input(shape=(784,), name="digits")
    x1 = keras.layers.Dense(64, activation="relu")(inputs)
    x2 = keras.layers.Dense(64, activation="relu")(x1)
    outputs = keras.layers.Dense(10, name="predictions")(x2)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


model = get_model()

# Prepare the training dataset.
batch_size = 32
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784)).astype("float32")
x_test = np.reshape(x_test, (-1, 784)).astype("float32")
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = Dataset(x_train, y_train, batch_size)

# Prepare the validation dataset.
val_dataset = Dataset(x_val, y_val, batch_size)


NameError: name 'Dataset' is not defined

In [6]:
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

In [7]:
def compute_loss_and_updates(trainable_variables, non_trainable_variables, x, y):
    y_pred, non_trainable_variables = model.stateless_call(
        trainable_variables, non_trainable_variables, x, training=True
    )
    loss = loss_fn(y, y_pred)
    return loss, non_trainable_variables
grad_fn = jax.value_and_grad(compute_loss_and_updates, has_aux=True)


In [8]:
small_batch = x_train[:128]
small_labels = y_train[:128]
tv = model.trainable_variables
ntv = model.non_trainable_variables
compute_loss_and_updates(tv, ntv, small_batch, small_labels)

(Array(71.96793, dtype=float32), [])

In [10]:
grad_fn(tv, ntv, small_batch, small_labels)

RecursionError: Exception encountered when calling Dense.call().

[1mmaximum recursion depth exceeded in comparison[0m

Arguments received by Dense.call():
  • inputs=jnp.ndarray(shape=(128, 784), dtype=float32)
  • training=True